In [1]:
import sys

sys.path.insert(0, "/Users/joerg/git_libs/pyiron_core")

In [2]:
import pyiron_workflow.graph.graph_operations as go
from pyiron_workflow import as_function_node
from pyiron_workflow.graph import base, gui
from pyiron_workflow.simple_workflow import Workflow
import pyiron_nodes as pn
# from pyironflow.pyironflow import PyironFlow

Why not running the graph rather than the workflow. Graph is much more powerful and includes serialization and subgraphs? This works already! Does not yet have concept of input.

In [3]:
def Subgraph(structure, ):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('subgraph')

    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure) 
    wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat) 
    wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure) 

    return wf.Repeat_2

structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
Subgraph(structure=structure).pull()

DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!


Al: [0. 0. 0.]
Al: [0.    2.025 2.025]
Al: [2.025 0.    2.025]
Al: [2.025 2.025 0.   ]
tags: 
    indices: [0 0 0 0]
pbc: [ True  True  True]
cell: 
Cell([4.05, 4.05, 4.05])

In [4]:
def my_func(structure=None, repeat_scalar=1):
    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow("subgraph")

    # wf.va_i_subgraph__structure = __main__.InputNode()
    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure, repeat_scalar=repeat_scalar)
    wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat)
    wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure)

    return wf.Repeat_2.pull()
   
structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
my_func(structure=structure, repeat_scalar=2);


DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!


In [5]:
structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
import pyiron_nodes

wf = Workflow("subgraph")

# wf.va_i_subgraph__structure = __main__.InputNode()
wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure)
wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat, repeat_scalar=1)
wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure)

wf.run()


DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!


Al: [0. 0. 0.]
Al: [0.    2.025 2.025]
Al: [2.025 0.    2.025]
Al: [2.025 2.025 0.   ]
tags: 
    indices: [0 0 0 0]
pbc: [ True  True  True]
cell: 
Cell([4.05, 4.05, 4.05])

## From Graph

In [6]:
import pyiron_nodes as pn


@as_function_node
def InputNode(structure=None):
    return structure


@as_function_node
def OutputNode(structure=None):
    return structure

InputStructure = "va_i_subgraph__structure"
OutputStructure = "va_o_subgraph__structure"

sub_graph = base.Graph(label="subgraph")
sub_graph += InputNode(label=InputStructure)
sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat")
sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat_2")
sub_graph += pn.atomistic.structure.view.Plot3d(label="Plot3d")
sub_graph += OutputNode(label=OutputStructure)

sub_graph += base.GraphEdge("Repeat", "Repeat_2", "structure", "structure")
sub_graph += base.GraphEdge(InputStructure, "Repeat", "structure", "structure")
sub_graph += base.GraphEdge(InputStructure, "Plot3d", "structure", "structure")
sub_graph += base.GraphEdge("Repeat_2", OutputStructure, "structure", "structure")

graph = base.Graph(label="Workflow")
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
graph += sub_graph
graph.nodes["subgraph"].expanded = False
graph.nodes["subgraph"].node = go.graph_to_node(sub_graph)

graph += pn.atomistic.structure.view.Plot3d(label="plot")

graph += base.GraphEdge("structure", "repeat", "structure", "structure")
graph += base.GraphEdge(OutputStructure, "plot", "structure", "structure")
graph += base.GraphEdge("repeat", InputStructure, "structure", "structure")
# graph += base.GraphEdge("subgraph", "plot", "structure", "structure")
# graph += base.GraphEdge("repeat", "subgraph", "structure", "structure")

graph = base.remove_node(graph, "Repeat")
graph = base.remove_node(graph, "Repeat_2")


base.GuiGraph(graph);

In [7]:
print(graph.nodes["subgraph"].expanded)
# Collapse a node
collapsed_graph = base.collapse_node(graph, "subgraph")

# Expand a node
# expanded_graph = go._expand_node(graph, "subgraph")
print(graph.nodes["subgraph"].expanded)


False
Collapsing node subgraph
False


In [8]:
# base.GuiGraph(expanded_graph);

In [9]:
base.GuiGraph(collapsed_graph);

In [10]:
graph.nodes["subgraph"].node.__getstate__() # correct function has to be constructed and stored in the node!

{'label': 'subgraph',
 'function': 'pyiron_workflow.graph.graph_operations.foo',
 'inputs': {}}

## From workflow

In [11]:
wf = Workflow('lammps_full')

# wf.structure = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.lammps = pn.atomistic.engine.lammps.Code1() # structure=wf.structure)
# wf.md_output = pn.atomistic.calculator.data.OutputCalcMD(dataclass=wf.lammps.outputs.generic)

In [12]:
graph = base.Graph(label='test')
graph += pn.atomistic.engine.lammps.Code1()
graph.nodes["Code1"].expanded = False

base.GuiGraph(graph)
                   

label:  LammpsMacro Code1
wf_label:  Code1
target:  Collect generic
target:  InitLammps path
new_graph:  Code1


ReactFlowWidget()

In [13]:
graph = base.get_full_graph_from_wf(wf)


label:  LammpsMacro lammps
wf_label:  lammps
target:  Collect generic
target:  InitLammps path
new_graph:  lammps


In [14]:
base.get_import_path_from_type(pn.atomistic.engine.lammps.Code1)

'pyiron_nodes.atomistic.engine.lammps.Code1_postfix'

In [15]:
# wf = Workflow("flow")
# wf.LoadGraph = pn.graphs.flow.LoadGraph('lammps_full')
# wf.DisplayNodes = pn.graphs.flow.DisplayNodes(graph=wf.LoadGraph)
# wf.DisplayNodes.pull()

# graph = base.get_full_graph_from_wf(wf)

In [16]:
# base.GuiGraph(pf.graph)

In [17]:
pf = gui.PyironFlow([wf, 'flow_expand', 'edges'])
pf.gui

label:  LammpsMacro lammps
wf_label:  lammps
target:  Collect generic
target:  InitLammps path
new_graph:  lammps
Expanding node lammps
Collapsing node Code1


In [18]:
pf.graph.nodes

AttributeError: 'PyironFlow' object has no attribute 'graph'

In [ ]:
xx

In [ ]:
label = 'lammps'
print(graph.nodes[label].expanded)
# Collapse a node
collapsed_graph = base.collapse_node(graph, label)
print(graph.nodes[label].expanded)

# Expand a node
expanded_graph = base.expand_node(graph, label)
print(graph.nodes[label].expanded)

In [ ]:
collapsed_graph.edges

In [ ]:
base.GuiGraph(collapsed_graph)

In [ ]:
base.GuiGraph(expanded_graph)

In [ ]:
base.GuiGraph(graph)

In [ ]:
graph.nodes

In [ ]:
collapsed_graph.nodes

In [ ]:
graph.nodes['lammps'].expanded = False

u_graph = base.get_updated_graph(graph)

graph.nodes['lammps'].expanded 



In [ ]:
base.GuiGraph(u_graph)